In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import os

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_2 = application_df.drop(['EIN', 'NAME'], axis=1).copy()

application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Choose a cutoff value and create a list of application types to be replaced

application_types_to_replace = []
app_counts = application_df_2['APPLICATION_TYPE'].value_counts()

for x in range(0,len(app_counts)):
    
    # adjusting cut off to 10000 from 250 in training 1
    if app_counts[x] < 10000:
        application_types_to_replace.append(app_counts.index[x])


# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
Other     7262
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []
class_counts = application_df_2['CLASSIFICATION'].value_counts()

for x in range(0,len(class_counts)):
    
    # adjusting cut off to 10000 from 250 in training 1
    if class_counts[x] < 2000:
        classifications_to_replace.append(class_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df_2)

application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_dim = len(X_train_scaled[0])

# First hidden layer
first_units = 2*input_dim
nn.add(tf.keras.layers.Dense(units=first_units, activation = 'relu', input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=first_units/2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=int(first_units/4), activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 68)                2380      
_________________________________________________________________
dense_1 (Dense)              (None, 34)                2346      
_________________________________________________________________
dense_2 (Dense)              (None, 17)                595       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 18        
Total params: 5,339
Trainable params: 5,339
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# create checkpoint callbacks to save model's weights
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='training_2/weights{epoch:04d}-acc{accuracy:.04f}.h5', 
                                                save_weights_only=True, period=5, monitor='accuracy')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 4s 143us/sample - loss: 0.5984 - accuracy: 0.6930
Epoch 2/50
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5837 - accuracy: 0.7081
Epoch 3/50
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5812 - accuracy: 0.7094
Epoch 4/50
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5798 - accuracy: 0.7094
Epoch 5/50
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5790 - accuracy: 0.7100
Epoch 6/50
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5783 - accuracy: 0.7105
Epoch 7/50
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5774 - accuracy: 0.7123
Epoch 8/50
25724/25724 [==============================] - 3s 112us/sample - loss: 0.5778 - accuracy: 0.7122
Epoch 9/50
25724/25724 [==============================] - 3s 112us/sample - loss: 0.5772 - accuracy: 0.7134
Epoch

In [12]:
# Export our model to HDF5 file
!mkdir -p training_2_saved_model
nn.save('training_2_saved_model/AlphabetSoupCharity')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: training_2_saved_model/AlphabetSoupCharity/assets


Trial 2, keep layers the same but keep dimensionality reduced

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_dim = len(X_train_scaled[0])

# First hidden layer
first_units = 86
nn.add(tf.keras.layers.Dense(units=first_units, activation = 'relu', input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=first_units/2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=int(first_units/4), activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 86)                3010      
_________________________________________________________________
dense_5 (Dense)              (None, 43)                3741      
_________________________________________________________________
dense_6 (Dense)              (None, 21)                924       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 22        
Total params: 7,697
Trainable params: 7,697
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='training_3/weights{epoch:04d}-acc{accuracy:.04f}.h5', 
                                                save_weights_only=True, period=5, monitor='accuracy')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 4s 137us/sample - loss: 0.5960 - accuracy: 0.6991
Epoch 2/50
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5847 - accuracy: 0.7070
Epoch 3/50
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5823 - accuracy: 0.7091
Epoch 4/50
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5802 - accuracy: 0.7109
Epoch 5/50
25724/25724 [==============================] - 3s 118us/sample - loss: 0.5795 - accuracy: 0.7118
Epoch 6/50
25724/25724 [==============================] - 3s 120us/sample - loss: 0.5786 - accuracy: 0.7117
Epoch 7/50
25724/25724 [==============================] - 3s 118us/sample - loss: 0.5778 - accuracy: 0.7117
Epoch 8/50
25724/25724 [==============================] - 3s 114us/sample - loss: 0.5774 - accuracy: 0.7115
Epoch 9/50
25724/25724 [==============================] - 3s 116us/sample - loss: 0.5766 - accuracy: 0.7131
Epoch

In [15]:
# Export our model to HDF5 file
!mkdir -p training_3_saved_model
nn.save('training_3_saved_model/AlphabetSoupCharity')

INFO:tensorflow:Assets written to: training_3_saved_model/AlphabetSoupCharity/assets


Trial 3, same results as trial 2. Reducing feature set seems to hurt performance

In [23]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_2 = application_df.drop(['EIN', 'NAME'], axis=1).copy()

application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Choose a cutoff value and create a list of application types to be replaced

application_types_to_replace = []
app_counts = application_df_2['APPLICATION_TYPE'].value_counts()

for x in range(0,len(app_counts)):
    
    # adjusting cut off to 10000 from 250 in training 1
    if app_counts[x] < 250:
        application_types_to_replace.append(app_counts.index[x])


# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []
class_counts = application_df_2['CLASSIFICATION'].value_counts()

for x in range(0,len(class_counts)):
    
    if class_counts[x] < 500:
        classifications_to_replace.append(class_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df_2)

# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_dim = len(X_train_scaled[0])

# First hidden layer
first_units = 3*input_dim
nn.add(tf.keras.layers.Dense(units=first_units, activation = 'relu', input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=first_units/2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=int(first_units/4), activation = 'relu'))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=int(first_units/8), activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 132)               5940      
_________________________________________________________________
dense_9 (Dense)              (None, 66)                8778      
_________________________________________________________________
dense_10 (Dense)             (None, 33)                2211      
_________________________________________________________________
dense_11 (Dense)             (None, 16)                544       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 17,490
Trainable params: 17,490
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='training_4/weights{epoch:04d}-acc{accuracy:.04f}.h5', 
                                                save_weights_only=True, period=5, monitor='accuracy')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 4s 165us/sample - loss: 0.5674 - accuracy: 0.7225
Epoch 2/50
25724/25724 [==============================] - 5s 179us/sample - loss: 0.5549 - accuracy: 0.7284
Epoch 3/50
25724/25724 [==============================] - 4s 168us/sample - loss: 0.5515 - accuracy: 0.7304
Epoch 4/50
25724/25724 [==============================] - 5s 206us/sample - loss: 0.5499 - accuracy: 0.7327
Epoch 5/50
25724/25724 [==============================] - 5s 206us/sample - loss: 0.5480 - accuracy: 0.7317
Epoch 6/50
25724/25724 [==============================] - 6s 231us/sample - loss: 0.5475 - accuracy: 0.7335
Epoch 7/50
25724/25724 [==============================] - 5s 191us/sample - loss: 0.5467 - accuracy: 0.7324
Epoch 8/50
25724/25724 [==============================] - 5s 178us/sample - loss: 0.5454 - accuracy: 0.7334
Epoch 9/50
25724/25724 [==============================] - 5s 185us/sample - loss: 0.5454 - accuracy: 0.7336
Epoch

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5277 - accuracy: 0.7284
Loss: 0.56611155934306, Accuracy: 0.728396475315094


Trial 4: still no good with extra neurons and more layers

In [35]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=10), activation=activation, input_dim=44))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=60,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [36]:
 # Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [37]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 57s]
val_accuracy: 0.7327113747596741

Best val_accuracy So Far: 0.7345772385597229
Total elapsed time: 01h 01m 33s
INFO:tensorflow:Oracle triggered exit


In [38]:
 # Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 71, 'num_layers': 4, 'units_0': 21, 'units_1': 26, 'units_2': 31, 'units_3': 31, 'units_4': 41, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '1ad42bd8a63c8275c9bfb1a15616923e'}
{'activation': 'relu', 'first_units': 61, 'num_layers': 1, 'units_0': 31, 'units_1': 6, 'units_2': 1, 'units_3': 11, 'units_4': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': 'ea802a1fdf34054dde25fff5b7f7b138'}
{'activation': 'relu', 'first_units': 81, 'num_layers': 2, 'units_0': 46, 'units_1': 6, 'units_2': 56, 'units_3': 56, 'units_4': 46, 'units_5': 46, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [39]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5292 - accuracy: 0.7346
Loss: 0.5509904949519099, Accuracy: 0.7345772385597229
8575/1 - 1s - loss: 0.5435 - accuracy: 0.7341
Loss: 0.5545167791878169, Accuracy: 0.7341107726097107
8575/1 - 1s - loss: 0.5440 - accuracy: 0.7336
Loss: 0.5522572382407007, Accuracy: 0.7336443066596985


Trial 5: Hyperparameter tuning using the tuner search method. Again, accuracy not improving... needs further investigation.